In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def _dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=_dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.countries,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

Post-loading: `Assignment`: 100%|██████████| 6/6 [00:00<00:00, 190.80it/s]


In [8]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db2 = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db2)

    stats = pstats.Stats(pr)

Post-loading: `Assignment`: 100%|██████████| 6/6 [00:00<00:00, 148.72it/s]


In [9]:
# db[User].extract()

In [10]:
db[User].keys()[0]

'2e9f4330-4782-452c-b432-7fd7222da999'

In [11]:
del db[User][db[User].keys()[0]]

In [12]:
db[User].df()

_id,name,age
str,str,i64
"""3309e510-ec50-453d-8bf0-a660da…","""Jack""",66
"""ab597087-0e58-47da-95c6-786979…","""Jane""",56


In [13]:
db2[User].df()

_id,name,age
str,str,i64
"""287d2b1b-7fd7-49bc-96d7-df0a6e…","""John""",26
"""b1676a83-2819-46b8-92cd-076dc4…","""Jack""",66
"""f45209dd-f10c-4328-adeb-411b6d…","""Jane""",36


In [14]:
db3 = db | db2
db3.describe()


{
    'contents': {
        'records': {
            'schema.User': 5,
            'schema.Organization': 6,
            'schema.Task': 12,
            'schema.Project': 3,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 14},
        'relations': {'schema.SearchResult': 3, 'schema.Membership': 14, 'schema.Assignment': 12}
    },
    'backend': None,
    'overlay': 'union/(a554f9a4e5|3f6f874528)'
}

In [15]:
x = db[Search.results]._link
assert x is not None
x.df()

schema.Search.term,schema.Search.results._rel._id,number,name,start,end,status,org__id
str,str,i64,str,date,date,str,str
"""test""","""104d65b581""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""37336059-9744-46ba-94ea-e1674f…"
"""test""","""11faab1a21""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""1a8142ce-c49e-45da-958b-10057c…"
"""test""","""14941c52d3""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""c88a49d4-ca76-4a58-965d-15a6c9…"


In [16]:
x = db[Search.results.x.tasks.x.assignees]
print(x.select_str)
x.keys()
x.df(sort_by=[User.name])

SELECT DISTINCT "schema.Search".term AS "schema.Search.term",
                "schema.Search.results".number AS "schema.Search.results.number",
                "schema.Search.results.tasks"._id AS "schema.Search.results.tasks._id",
                "schema.Task.assignees"._id AS "schema.Search.results.tasks.assignees._id",
                "schema.Task.assignees"._id AS _id,
                "schema.Task.assignees".name AS name,
                "schema.Task.assignees".age AS age
FROM "schema_Search" AS "schema.Search"
JOIN "schema_SearchResult" AS "schema.Search.results._rel" ON "schema.Search.results._rel"._from_term = "schema.Search".term
JOIN "schema_Project" AS "schema.Search.results" ON "schema.Search.results._rel"._to_number = "schema.Search.results".number
JOIN "schema_Task" AS "schema.Search.results.tasks" ON "schema.Search.results.tasks".project_number = "schema.Search.results".number
JOIN "schema_Assignment" AS "schema.Search.results.tasks.assignees._rel" ON "schema.Search.resul

schema.Search.term,schema.Search.results.number,schema.Search.results.tasks._id,_id,name,age
str,i64,str,str,str,i64
"""test""",1,"""7d955d57-2454-4d40-b5c6-0bdc1e…","""ab597087-0e58-47da-95c6-786979…","""Jane""",56
"""test""",0,"""cdd66bdf-229e-4639-b4f8-24152a…","""ab597087-0e58-47da-95c6-786979…","""Jane""",56


In [17]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev152+g2c0bcb5.d20250123',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': '3ae3cfc867398c720b692e3374b796e1c6600fe7',
        'module_path': WindowsPath('exp/schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'schema.User': 2,
            'schema.Organization': 3,
            'schema.Task': 6,
            'schema.Project': 3,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 7},
        'relations': {'schema.SearchResult': 3, 'schema.Membership': 7, 'schema.Assignment': 6}
    },
    'backend': None
}

In [18]:
nodes, edges = db.to_graph([Search, Project._table[Project.number == 1], Organization])

In [19]:
x = db[Project.number]

In [20]:
nodes

,node_id,table,schema.Search.term,schema.Search.result_count,schema.Project.number,schema.Project.name,schema.Project.start,schema.Project.end,schema.Project.status,schema.Project.org__id,schema.Organization._id,schema.Organization.name,schema.Organization.address,schema.Organization.city
0,0,schema_Search,test,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,schema_Project,NaN,NaN,1.0,cleaning shoes,2020-01-02,2020-01-05,done,1a8142ce-c49e-45da-958b-10057cd6c218,NaN,NaN,NaN,NaN
2,2,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1a8142ce-c49e-45da-958b-10057cd6c218,Shoe Shop,Main Street 2,Shoetown
3,3,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37336059-9744-46ba-94ea-e1674f52b1ed,Car Shop,Main Street 3,Cartown
4,4,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c88a49d4-ca76-4a58-965d-15a6c9a2a767,Bakery,Main Street 1,Bakerville


In [21]:
edges

,source,target,ltr,rtl,schema.SearchResult._to_number,schema.SearchResult._id,schema.SearchResult._from_term,schema.SearchResult.score
0,1,2,org__id,None,NaN,NaN,NaN,NaN
1,0,1,_to_number,_from_term,1.0,11faab1a21,test,0.8


In [22]:
db[User].values()


[
    User({'_id': '3309e510-ec50-453d-8bf0-a660daa51d2b', 'name': 'Jack', 'age': 66}),
    User({'_id': 'ab597087-0e58-47da-95c6-786979734cb5', 'name': 'Jane', 'age': 56})
]

In [23]:
db[Project].df()

number,name,start,end,status,org__id
i64,str,date,date,str,str
0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""c88a49d4-ca76-4a58-965d-15a6c9…"
1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""1a8142ce-c49e-45da-958b-10057c…"
2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""37336059-9744-46ba-94ea-e1674f…"


In [24]:
db[Search.term].values()

['test']

In [25]:
list(db[Search.result_count].items())

[('test', 3)]

In [26]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 0, 'c88a49d4-ca76-4a58-965d-15a6c9a2a767'),
        Organization({'_id': 'c88a49d4-ca76-4a58-965d-15a6c9a2a767', 'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville'})
    ),
    (
        ('test', 1, '1a8142ce-c49e-45da-958b-10057cd6c218'),
        Organization({'_id': '1a8142ce-c49e-45da-958b-10057cd6c218', 'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown'})
    ),
    (
        ('test', 2, '37336059-9744-46ba-94ea-e1674f52b1ed'),
        Organization({'_id': '37336059-9744-46ba-94ea-e1674f52b1ed', 'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown'})
    )
]

In [27]:
db[Search.results].items()


[
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    ),
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    ),
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    )
]

In [28]:
list(db[Search.results]._abs_cols.keys())

['number', 'name', 'start', 'end', 'status', 'org__id']

In [29]:
db[Search.results].df()

schema.Search.term,number,name,start,end,status,org__id
str,i64,str,date,date,str,str
"""test""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""c88a49d4-ca76-4a58-965d-15a6c9…"
"""test""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""1a8142ce-c49e-45da-958b-10057c…"
"""test""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""37336059-9744-46ba-94ea-e1674f…"


In [30]:
db[Search.results.x.org].df()

schema.Search.term,schema.Search.results.number,_id,name,address,city
str,i64,str,str,str,str
"""test""",0,"""c88a49d4-ca76-4a58-965d-15a6c9…","""Bakery""","""Main Street 1""","""Bakerville"""
"""test""",1,"""1a8142ce-c49e-45da-958b-10057c…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""test""",2,"""37336059-9744-46ba-94ea-e1674f…","""Car Shop""","""Main Street 3""","""Cartown"""


In [31]:
db[Organization].df()

_id,name,address,city
str,str,str,str
"""1a8142ce-c49e-45da-958b-10057c…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""37336059-9744-46ba-94ea-e1674f…","""Car Shop""","""Main Street 3""","""Cartown"""
"""c88a49d4-ca76-4a58-965d-15a6c9…","""Bakery""","""Main Street 1""","""Bakerville"""


In [32]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_id,score,_from_term,_to_number
str,f64,str,i64
"""14941c52d3""",0.9,"""test""",0


In [33]:
import pandas as pd


db[Organization.countries].df(pd.DataFrame)

value
schema.Organization._id              schema.Organization.countries.idx        
1a8142ce-c49e-45da-958b-10057cd6c218 0                                     USA
                                     1                                  Mexico
37336059-9744-46ba-94ea-e1674f52b1ed 0                                     USA
                                     1                                  Canada
c88a49d4-ca76-4a58-965d-15a6c9a2a767 0                                     USA
                                     1                                  Mexico
                                     2                                  Canada

In [34]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         540397 function calls (487324 primitive calls) in 1.217 seconds

   Ordered by: cumulative time
   List reduced from 1771 to 156 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       11    0.000    0.000    0.506    0.046 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:847(load)
       55    0.001    0.000    0.493    0.009 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:696(_load_rec_from_item)
       55    0.002    0.000    0.491    0.009 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:568(_load_record)
    36/12    0.001    0.000    0.485    0.040 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:661(_load_records)
       18    0.000    0.000    0.478    0.027 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1669(__ior__)
       18    0.002    0.000    0.478    0.027 C:\Users\lworm\Repos\py-research\src\py_research\db\databas

   0.001    0.000    0.033    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2155(_generic_type)
     2516    0.008    0.000    0.032    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2600(_hint_to_typedef)
       48    0.000    0.000    0.028    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:536(_gen_match_expr)
       18    0.002    0.000    0.027    0.002 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2960(_gen_upload_table)
       47    0.000    0.000    0.027    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:551(<genexpr>)
      382    0.000    0.000    0.026    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:547(value_type)
        1    0.000    0.000    0.024    0.024 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3154(_mutate_from_values)
     3137    0.003    0.000    0.023    0.000 C:\Users\lworm\Repos\py-research\src\p